In [1]:
import data_smoothing
import find_peaks
import peak_character
import peak_classify

# Load real TA data

In [2]:
data_nm, data_time, data_z = data_smoothing.load_data_csv('exp03_20180103 -t0 -chirp.csv')

# Smoothen data

In [3]:
smooth_matx = data_smoothing.earth_smooth_matrix(data_nm, data_z)

/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [14]:
smooth_matx_np = np.array(smooth_matx)

In [23]:
import numpy as np
import peakutils
from peakutils.plot import plot as pplot
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy import interpolate
import pandas as pd



def indexes(y, thres=0.3, min_dist=1):
    """Peak detection routine.

    Finds the numeric index of the peaks in *y* by taking its first order difference. By using
    *thres* and *min_dist* parameters, it is possible to reduce the number of
    detected peaks. *y* must be signed.

    Parameters
    ----------
    y : ndarray (signed)
        1D amplitude data to search for peaks.
    thres : float between [0., 1.]
        Normalized threshold. Only the peaks with amplitude higher than the
        threshold will be detected.
    min_dist : int
        Minimum distance between each detected peak. The peak with the highest
        amplitude is preferred to satisfy this constraint.

    Returns
    -------
    ndarray
        Array containing the numeric indexes of the peaks that were detected
    """
    y_raw = y
    y = [abs(k) for k in y_raw]
    
    if isinstance(y, np.ndarray) and np.issubdtype(y.dtype, np.unsignedinteger):
        raise ValueError("y must be signed")

    thres = thres * (np.max(y) - np.min(y)) + np.min(y)
    min_dist = int(min_dist)

    # compute first order difference
    dy = np.diff(y)

    # propagate left and right values successively to fill all plateau pixels (0-value)
    zeros,=np.where(dy == 0)
    
    # check if the singal is totally flat
    if len(zeros) == len(y) - 1:
        return np.array([])
    
    while len(zeros):
        # add pixels 2 by 2 to propagate left and right value onto the zero-value pixel
        zerosr = np.hstack([dy[1:], 0.])
        zerosl = np.hstack([0., dy[:-1]])

        # replace 0 with right value if non zero
        dy[zeros]=zerosr[zeros]
        zeros,=np.where(dy == 0)

        # replace 0 with left value if non zero
        dy[zeros]=zerosl[zeros]
        zeros,=np.where(dy == 0)

    # find the peaks by using the first order difference
    peaks = np.where((np.hstack([dy, 0.]) < 0.)
                     & (np.hstack([0., dy]) > 0.)
                     & (y > thres))[0]

    # handle multiple peaks, respecting the minimum distance
    if peaks.size > 1 and min_dist > 1:
        highest = peaks[np.argsort(y[peaks])][::-1]
        rem = np.ones(y.size, dtype=bool)
        rem[peaks] = False

        for peak in highest:
            if not rem[peak]:
                sl = slice(max(0, peak - min_dist), peak + min_dist + 1)
                rem[sl] = True
                rem[peak] = False

        peaks = np.arange(y.size)[~rem]
    
    
    return peaks

In [24]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [25]:
 # get peak height and fwhm info
def peakchar(data_nm, data_z_array, peak_index):
    """find the peak width, and intensity"""
    num_peaks = len(peak_index)
    
    #array of peak height
    height = [data_z_array[idx] for idx in peak_index]
    
    #array of peak width
    half_height = [ht / 2 for ht in height]

    fwhm_idx_1 = np.empty_like(half_height)
    fwhm_idx_2 = np.empty_like(fwhm_idx_1)
    fwhm_nm_1 = np.empty_like(fwhm_idx_1)
    fwhm_nm_2 = np.empty_like(fwhm_idx_1)
    
    for i in range(num_peaks):
        #find the index and nmof the left side of the fwhm
        if i == 0:
            fwhm_idx_1[i] = find_nearest(data_z_array[0:peak_index[i]], half_height[i])
        else:
            fwhm_idx_1[i] = find_nearest(data_z_array[peak_index[i-1]:peak_index[i]], half_height[i]) + peak_index[i-1]

        fwhm_nm_1[i] = data_nm[int(fwhm_idx_1[i])]
        
        #find the index and nm of the right side of the fwhm   
        fwhm_idx_2[i] = find_nearest(data_z_array[peak_index[i]:], half_height[i]) + peak_index[i]

        fwhm_nm_2[i] = data_nm[int(fwhm_idx_2[i])]
    
    #find fwhm
    fwhm = fwhm_nm_2 - fwhm_nm_1

    return height, fwhm

In [26]:
def peak_matrix(nm_array,data_matrix, threshold, mindist):
    """find peaks in a data matrix
    and calculate the height and width of the peaks"""
    
    peak_idx_matx = []
    peak_height_matx = []
    peak_fwhm_matx = []
    
    num_timeslice = np.shape(data_matrix)[1]
        

    for i in range(num_timeslice):
        data_timeslice = data_matrix[:, i]
        
        peak_idx = indexes(data_timeslice, threshold, mindist).tolist()
        peak_idx_matx.append(peak_idx)
        
        
        peak_height, peak_fwhm = peakchar(nm_array, data_timeslice, peak_idx)
        
        peak_height_matx.append(peak_height)
        peak_fwhm_matx.append(peak_fwhm)
        
        # transfer to dataframe
        peak_idx_df=pd.DataFrame(peak_idx_matx)
        peak_height_df=pd.DataFrame(peak_height_matx)
        peak_fwhm_df=pd.DataFrame(peak_fwhm_matx)
        
    return peak_idx_df, peak_height_df, peak_fwhm_df


# Find peaks

## I ran the above functions directly, rather than the ones in .py files because I was trouble-shooting the code

## i.e. note that I ran *peak_matrix* rather than *character.peak_matrix*

## i didn't change the function so you can run the same function from import as well

In [27]:
threshold = 0
mindist = 0
idx, height, fwhm = peak_matrix(data_nm,smooth_matx_np, threshold, mindist)

# Two peaks were identified: index ~17 and index ~54

In [32]:
print (idx[100:200])

        0      1      2   3
100  16.0   96.0    NaN NaN
101  16.0    NaN    NaN NaN
102  17.0    NaN    NaN NaN
103  17.0    NaN    NaN NaN
104  17.0    NaN    NaN NaN
105  17.0    NaN    NaN NaN
106  17.0    NaN    NaN NaN
107  17.0    NaN    NaN NaN
108  17.0    NaN    NaN NaN
109  17.0    NaN    NaN NaN
110  17.0    NaN    NaN NaN
111  17.0    NaN    NaN NaN
112  16.0    NaN    NaN NaN
113  17.0    NaN    NaN NaN
114  17.0    NaN    NaN NaN
115  17.0  114.0    NaN NaN
116  17.0    NaN    NaN NaN
117  17.0    NaN    NaN NaN
118  17.0  111.0    NaN NaN
119  17.0    NaN    NaN NaN
120  17.0    NaN    NaN NaN
121  17.0    NaN    NaN NaN
122  17.0    NaN    NaN NaN
123  17.0    NaN    NaN NaN
124  16.0    NaN    NaN NaN
125  17.0    NaN    NaN NaN
126  17.0    NaN    NaN NaN
127  17.0    NaN    NaN NaN
128  17.0    NaN    NaN NaN
129  17.0    NaN    NaN NaN
..    ...    ...    ...  ..
170  17.0   51.0    NaN NaN
171  17.0   51.0    NaN NaN
172  17.0   48.0    NaN NaN
173  17.0   46.0    

In [34]:
corrected_output = peak_classify.data_grouping(idx, height, fwhm)

In [35]:
peak_dict = peak_classify.cluster_classifier(idx, corrected_output)

# Here, after clustering, there is only one peak left... :(

In [36]:
peak_dict

{'peak_0': [], 'peak_1': [], 'peak_2': [Position     51.000000
  Height       -0.001273
  Width       148.110000
  Time        196.000000
  Name: 785, dtype: float64], 'peak_3': []}